In [9]:
from datetime import date
import calendar

key_array = [ 'January', 'February', 'March', 'April', 'May', 'June', 'July',
              'August', 'September', 'October', 'November', 'December' ]
month_length = [ 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]


def print_month(month, year):
    idx = key_array.index(month)
    day = 1
    
    wd = date(year, idx+1, day).weekday()
    wd = (wd+1) % 7
    end = month_length[idx]
    
    if calendar.isleap(year) and idx == 1:    # February LeapYear has one extra day
        end += 1

    print('{} {}'.format(month,year).center(20))
    print('Su Mo Tu We Th Fr Sa')
    print('   ' * wd, end='')                 # Pad spacing
    while day <= end:
        print('{:2d} '.format(day), end='')
        wd = (wd + 1) % 7
        day += 1
        if wd == 0: print()
    print()

In [15]:
print_month('November', 2003)

   November 2003    
Su Mo Tu We Th Fr Sa
                   1 
 2  3  4  5  6  7  8 
 9 10 11 12 13 14 15 
16 17 18 19 20 21 22 
23 24 25 26 27 28 29 
30 


In [19]:
days_in_month = {
    'January'   : 31,    'February'  : 28,   'March'     : 31,
    'April'     : 30,    'May'       : 31,   'June'      : 30,
    'July'      : 31,    'August'    : 31,   'September' : 30,
    'October'   : 31,    'November'  : 30,   'December'  : 31
}


class Entry:
    def __init__(self, k, v) -> None:
        self.key = k
        self.value = v

class HashTable:
    def __init__(self, M=10) -> None:
        self.table = [None] * M
        self.M = M
    
    def get(self, k):
        hc = hash(k) % self.M
        return self.table[hc].value if self.table[hc] else None
    
    def put(self, k, v):
        hc = hash(k) % self.M
        entry = self.table[hc]
        if entry:
            if entry.key == k:
                entry.value = v
            else:
                raise RuntimeError(f'Key Collision: {k} and {entry.key}')
        else:
            self.table[hc] = Entry(k, v)

In [27]:
table = HashTable(1000)
table.put('April', 30)
table.put('May', 31)

print(table.get('August'))
print(table.get('May'))

None
31


In [26]:
print(table.table)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non